# Исследование надежности заемщиков

### Описание проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Описание данных
Данные предоставлены в файле `loaners.csv`

#### Признаки:
Описание данных
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

### Задача
Предоставить результаты исследования, которые будут учтены при построении модели кредитного скоринга: какие признаки значительно влияют на невозврат кредита.

### Ход проекта
Проект будет выполнен в соответствии со следующими шагами:
1. Загрузка и предобработка данных
2. Анализ данных
3. Общий вывод

## 1. Загрузка и подготовка данных

In [1]:
# импорт необходимых библиотек
import pandas as pd

In [2]:
try:
    data = pd.read_csv('/datasets/loaners.csv')
except:
    data = pd.read_csv('datasets/loaners.csv')
    
display(data.head())
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Предобработка данных
#### Обработка пропусков и аномалий

In [3]:
# удаление пропусков
print('Обнаружены пропуски:')
data.isna().sum()

Обнаружены пропуски:


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски в столбце `total_income` заполним медианным значениям внутри каждого типа дохода. Такому выбору есть две причины: доходы, полученные разными способами могут значительно различаться (например, обычно доход пенсионера ниже дохода работающего человека). Именно медиана выбрана потому, что она меньше подвержена влиянию аномальных (слишком больших или маленьких значений), а потому более корректно, по сравнению со средним, отражает доход группы клиентов.

In [4]:
for in_type in data['income_type'].unique():
    data.loc[(data['income_type'] == in_type) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == in_type), 'total_income'].median()
    
print('Completed!')

Completed!


В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Сделаем отрицательные значения положительными.

In [5]:
# обрбаботка аномали
data.loc[data['days_employed'] < 0, 'days_employed'] = abs(data.loc[data['days_employed'] < 0, 'days_employed'])
print('Completed!')

Completed!


После корректировки данных в столбце 'days_employed' можем заполнить пропуски в этом столбце медианным значением по типу дохода.

In [6]:
for in_type in data['income_type'].unique():
    data.loc[(data['income_type'] == in_type) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == in_type), 'days_employed'].median()
    
print('Completed!')

Completed!


In [7]:
# аномалии в столбце 'children'
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [8]:
# удалим строки с аномальными значениями столбца 'children'
data = data.loc[(data['children'] != data['children'].unique()[4]) & (data['children'] != data['children'].unique()[6])]
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

#### Изменение типов данных
Заменим вещественный тип данных в столбце 'days_employed' на целочисленный, а данные столбца 'total_income' округлим до двух знаков после запятой.

In [9]:
data['total_income'] = round(data['total_income'], 2)
data['days_employed'] = data['days_employed'].astype(int)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу


In [10]:
data['family_status'] = data['family_status'].str.lower()  

In [11]:
# проверим на корректность и дубликаты столбец 'education'
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [12]:
# исправляем регистр
data['education'] = data['education'].str.lower()
print('Дубликатов обнаружено:', data.duplicated().sum())
data = data.drop_duplicates()
print('Все дубликаты удалены')

Дубликатов обнаружено: 71
Все дубликаты удалены


#### Категоризация данных
Создадим столбцы, которые будут обобщать информацию о доходах и целях займа.

Категоризируем доходы заемщиков, согласно списку ниже:
- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [13]:
def categorize_income(income):
    
    # принимает сумму дохода и возвращает его категорию
    
    if 0 <= income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    return 'A'

In [14]:
data['total_income_category'] = data['total_income'].apply(categorize_income)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья,B
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля,C
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья,C
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование,B
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу,C


На основании столбца `purpose` сформируем новый столбец `purpose_category`, котрорый укажает категорию назначения займа:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [15]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [16]:
def categorize_purpose(purpose):
    
#     приншимает строку и возвращает категорию назначения займа
    
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if ('жиль' in purpose) or ('недвижимост' in purpose):
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
         return 'получение образования'
    return 'другие цели'

In [17]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья,B,операции с недвижимостью
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья,C,операции с недвижимостью
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование,B,получение образования
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу,C,проведение свадьбы


### Вывод по разделу
Были заполнены пропуски, обработаны аномалии, преобразованы типы данных и удалены дубликаты.

Для дальнейшего анализа были созданы категоризирующие столбцы.

## 2. Анализ данных
В этом разделе попробуем ответить на вопрос, как разные признаки заказчиков влияют на возврат кредита в срок.

### Есть ли зависимость между количеством детей и возвратом кредита в срок?
Чтобы ответить на этот вопрос, посмотрим, сколько должников в каждой группе по количеству детей, а также общее количество заемщиков в каждой из категорий и надем их процент.

In [18]:
children_debt = data.groupby('children').agg({'debt': ['count', 'sum']})
children_debt.columns = ['К-во заемщиков', 'К-во неплательщиков']
children_debt['% неплательщиков'] = round(100*children_debt['К-во неплательщиков'] / children_debt['К-во заемщиков'], 2)
children_debt.sort_values(by='% неплательщиков', ascending=False)

,К-во заемщиков,К-во неплательщиков,% неплательщиков
children,,,
4,41,4,9.76
2,2052,194,9.45
1,4808,444,9.23
3,330,27,8.18
0,14091,1063,7.54
5,9,0,0.00


Что получилось:
- Похоже, что чаще всего не возвращают кредиты люди с 4-мя детьми (примерно 9,7%), хотя не очень понятно, насколько корректно брать в расчет эту категорию, потому что она очень малочисленная.
- Затем идут с 2-мя и 1-м ребенком (9,5% и 9,2%), а также 3-мя (8,2%)
- При этом люди без детей, у которых было самое большое количество долгов по кредиту, в процентном отношении возвращают их чаще (7,5 %), а люди с 5-ю детьми вообще идеальные кредитополучатели и никогда не просрочивают выплаты (но таких семей в принципе достаточно мало)

Похоже, что нет прямой зависимости между ростом количества детей и процентом задолженностей по кредиту.

Попробем категоризировать заемщиков по количеству детей, возможно, картина изменится. В России семьи по количеству детей можно группировать следующим образом:
- 0 - нет детей
- 1-2 ребенка
- 3 и больше - многодетные семьи

Сгруппируем данные и снова найдем процент долгов по кредитам.

In [19]:
def children_group(children):
    
    if children == 0:
         return 'нет детей'
    elif children <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетная семья'
    
data_kids = data.copy()
data_kids['children_group'] = data_kids['children'].apply(children_group)

grouped_children_debt = data_kids.groupby('children_group').agg({'debt': ['count', 'sum']})
grouped_children_debt.columns = ['К-во заемщиков', 'К-во неплательщиков']
grouped_children_debt['% неплательщиков'] = round(100*grouped_children_debt['К-во неплательщиков'] 
                                                  / grouped_children_debt['К-во заемщиков'], 2)
grouped_children_debt.sort_values(by='% неплательщиков', ascending=False)

,К-во заемщиков,К-во неплательщиков,% неплательщиков
children_group,,,
1-2 ребенка,6860,638,9.30
многодетная семья,380,31,8.16
нет детей,14091,1063,7.54


Похоже, что вывод остается таким же - нет зависимости между ростом количества детей в семье и процентом просрочки кредита.
Однако можно отметить, что разница в группах прослеживается и семьи с 1-2 детьми имеют проблемы с платежами немного чаще остальных, при этом заемщики без детей имеют самый низкий процент. Возможно, этот параметр нужно учесть при решении о выдаче кредита.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?
Для ответа на этот вопрос, сгруппируем данные по семейному положению и вычислим долю должников в для каждой категории.

In [20]:
family_debt = data.groupby('family_status').agg({'debt': ['count', 'sum']})
family_debt.columns = ['К-во заемщиков', 'К-во неплательщиков']
family_debt['% неплательщиков'] = round(100*family_debt['К-во неплательщиков'] / family_debt['К-во заемщиков'], 2)
family_debt.sort_values(by='% неплательщиков', ascending=False)

,К-во заемщиков,К-во неплательщиков,% неплательщиков
family_status,,,
не женат / не замужем,2796,273,9.76
гражданский брак,4134,385,9.31
женат / замужем,12261,927,7.56
в разводе,1189,84,7.06
вдовец / вдова,951,63,6.62


Какие выводы можно сделать:

- Неженатные/незамужние люди и люди, состоящии в гражданском браке имеют долги по кредитам в 9,7% и 9,3% случаев
- Люди в браке и в разводе имеют долги по кредитам в 7,5% и 7,1% случаев
- Вдовцы и вдовы имеют самые низкие проценты долгов по кредитам - 6,6%

Разница между категориями есть, и возможно, нужно учитывать семейное положение наряду с другими факторами при решении о выдаче кредита.

####  Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Как и в предыдущих вопросах, найдем общее количество заемщиков в этих категории, количество должников, и вычислим процент задолженностей по кредитам. Уровень дохода будем рассматривать в категориях, созданных в предыдущем отделе.

In [21]:
income_category_debt = data.groupby('total_income_category').agg({'debt': ['count', 'sum']})
income_category_debt.columns = ['К-во заемщиков', 'К-во неплательщиков']
income_category_debt['% неплательщиков'] = round(100*income_category_debt['К-во неплательщиков'] 
                                                 / income_category_debt['К-во заемщиков'], 2)
income_category_debt.sort_values(by='% неплательщиков', ascending=False)

,К-во заемщиков,К-во неплательщиков,% неплательщиков
total_income_category,,,
E,22,2,9.09
C,15921,1353,8.50
A,25,2,8.00
B,5014,354,7.06
D,349,21,6.02


Можно отметить, что количество заемщиков в категориях A и E очень маленькое относительно остальных категорий и общего числа, заемщики в категории D тоже немногочисленны. Посчитаем сколько процентов от общего числа заемщиков представлено каждой группой.

In [22]:
all_categories = income_category_debt['К-во заемщиков'].sum()

income_category_debt['Доля в общем к-ве'] = income_category_debt['К-во заемщиков'] / all_categories
income_category_debt.sort_values(by='Доля в общем к-ве', ascending=False)

,К-во заемщиков,К-во неплательщиков,% неплательщиков,Доля в общем к-ве
total_income_category,,,,
C,15921,1353,8.50,0.746379
B,5014,354,7.06,0.235057
D,349,21,6.02,0.016361
A,25,2,8.00,0.001172
E,22,2,9.09,0.001031


Выходит, что примерно 98% всех заемщиков представлено категориями B и С, а значит, по ним можно сделать вывод о платежеспобсности.

In [23]:
income_category_debt = data[(data['total_income_category'] == 'B') | (data['total_income_category'] == 'C')]. \
groupby('total_income_category').agg({'debt': ['count', 'sum']})
income_category_debt.columns = ['К-во заемщиков', 'К-во неплательщиков']
income_category_debt['% неплательщиков'] = round(100*income_category_debt['К-во неплательщиков'] 
                                                 / income_category_debt['К-во заемщиков'], 2)
income_category_debt.sort_values(by='% неплательщиков', ascending=False)

,К-во заемщиков,К-во неплательщиков,% неплательщиков
total_income_category,,,
C,15921,1353,8.50
B,5014,354,7.06


Согласно полученной таблице, у заемщиков в категории C доля должников 8.5%, а у категории В - 7%. Получается, что люди с более высоким доходом немного чаще оплачивают кредиты в срок. Значит, этот параметр может влиять на решение об одобрении кредита.

####  Как разные цели кредита влияют на его возврат в срок?
Посчитаем количество займов в каждой категории, количество должников и их процент.

In [24]:
purpose_category_debt = data.groupby('purpose_category').agg({'debt': ['count', 'sum']})
purpose_category_debt.columns = ['К-во заемщиков', 'К-во неплательщиков']
purpose_category_debt['% неплательщиков'] = round(100*purpose_category_debt['К-во неплательщиков'] 
                                                 / purpose_category_debt['К-во заемщиков'], 2)
purpose_category_debt.sort_values(by='% неплательщиков', ascending=False)

,К-во заемщиков,К-во неплательщиков,% неплательщиков
purpose_category,,,
операции с автомобилем,4279,400,9.35
получение образования,3988,369,9.25
проведение свадьбы,2313,183,7.91
операции с недвижимостью,10751,780,7.26


Исходя из таблицы наибольшее количество просрочек по кредиту у заемщиков, запрашивающих кредит на *операции с автомобилем* и *получение образования* - около 9,3%.
Заемщики, запрашивающие кредит на *проведение свадьбы* не возвращают его в срок в 7,9% случаев.
Меньше всего проблем у категории заемщиков, берущих кредит на *операции с недвижимостью* - около 7,3%.

Можно сделать вывод, что разница в доле должников есть, следовательно, нужно учитывать этот фактор при решении о выдаче кредита.

## 4. Общий вывод
В ходе выполнения проекта я оценила влияние различных факторов на платежеспособность заемщиков, рассмотрела:
- количество детей
- семейное положение
- доход
- цель кредита

Какие зависимости удалось обнаружить.

**Количество детей:** не дает прямого влияния на платежеспособность, однако разница выделенных категориях прослеживается: семьи с 1-2 детьми имеют проблемы с платежами немного чаще остальных, при этом заемщики без детей имеют самый низкий процент. Возможно, имеет смысл учесть этот параметр при решении об одобрении кредита.

**Семейное положение:** здесь можно сделать вывод, что люди состоящие/состоявшие в браке чаще возвращают кредит вовремя. Препдоложу, что этот параметр нужно учитывать при решении об одобрении кредита.

**Доход:** примечательным моментом при анализе этого параметра оказалось неравномерное распределение, в котором около 98% всех заемщиков представлено всего двумя категориями дохода (из пяти). Анализ этих двух категорий показал влияние роста дохода на платежеспособноть по кредиту (подтверждая интуитивное предположение)), значит, его 

**Цель кредита:** здесь разнообразные цели удалось категоризировать и выделить несколько групп, в которых наблюдается разница в частоте возврата кредита вовремя. Значит, нужно учитывать выделенные категории при решении об одобрении кредита.

Подытоживая, отмечу, что каждый из рассмотренных параметров может так или иначе влиять на платежеспособность, разница в том, что некоторые категории влияют прямо (как например доход или семейное положение), в других - разница прослеживается от категории к категории.